In [1]:
!pwd

/home/sagemaker-user/Brain-Tumor-Classification-MRI/notebooks


In [2]:
%cd ..

/home/sagemaker-user/Brain-Tumor-Classification-MRI


# Training Configuration

In [3]:
from dataclasses import dataclass

In [4]:
@dataclass
class TrainingConfig:
    training_folder: str
    trained_model_path: str
    updated_model_path: str
    training_data: str
    params_epochs: int
    params_batch_size: int
    params_augmentation: bool
    params_image_size: list

# Configuration Manager

In [5]:
from src.config import ConfigManager
from src.constants import *
from src.utils import create_dirs
import os

In [6]:
class ConfigManager(ConfigManager):
    def get_training_config(self):
        training_config = self.config.training
        model_config = self.config.model

        create_dirs([training_config.folder_name])
        
        training_data_path = os.path.join(
            self.artifacts_folder, self.config.data.data_folder_name, 'Training'
        )
        
        return TrainingConfig(
            training_folder = os.path.join(self.artifacts_folder, training_config.folder_name),
            trained_model_path = os.path.join(self.artifacts_folder, training_config.trained_model_path),
            updated_model_path = os.path.join(self.artifacts_folder, self.config.model.updated_model_path),
            training_data = training_data_path,
            params_epochs = self.params.EPOCHS,
            params_batch_size = self.params.BATCH_SIZE,
            params_augmentation = self.params.AUGMENTATION,
            params_image_size = self.params.IMAGE_SIZE
        )

# Train Function

In [7]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

2024-09-28 16:15:41.581235: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-28 16:15:41.608962: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
class Train:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def data_generator(self):
        if self.config.params_augmentation:
            train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale = 1/255.,
                validation_split = 0.27,
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2
            )
        else:
            train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale = 1/255.,
                validation_split = 0.27
            )
        self.train_generator = train_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'training',
            shuffle = True,
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size
        )

        val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale = 1/255.,
            validation_split = 0.27
        )
        self.val_generator = val_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = True,
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size
        )

    def fit(self, callbacks: list):
        model = tf.keras.models.load_model(self.config.updated_model_path)
        steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        val_steps = self.val_generator.samples // self.val_generator.batch_size

        model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = steps_per_epoch,
            validation_data = self.val_generator,
            validation_steps = val_steps,
            callbacks = callbacks
        )

        model.save(self.config.trained_model_path)

# Perform The Step

In [9]:
from src.components.callbacks import Callbacks

In [10]:
try:
    callbacks_config = ConfigManager().get_callbacks_config()
    callbacks = Callbacks(callbacks_config).get_callbacks()
    
    training_config = ConfigManager().get_training_config()
    train = Train(training_config)
    train.data_generator()
    train.fit(callbacks)
except Exception as e:
    raise e

[2024-09-28 16:15:43,294: INFO: utils: The file: config.yaml loaded successfully...]
[2024-09-28 16:15:43,296: INFO: utils: The file: params.yaml loaded successfully...]
[2024-09-28 16:15:43,307: INFO: utils: The file: config.yaml loaded successfully...]
[2024-09-28 16:15:43,308: INFO: utils: The file: params.yaml loaded successfully...]
Found 2097 images belonging to 4 classes.
Found 773 images belonging to 4 classes.
65/65 [==============================] - 260s 4s/step - loss: 1.6213 - accuracy: 0.2780 - val_loss: 1.3777 - val_accuracy: 0.2891


# Tensorboard Logs

#### If you want to see the logs, run the following terminal command:
`tensorboard --logdir artifacts/callbacks/tensorboard`

In [13]:
!tensorboard --logdir artifacts/callbacks/tensorboard --bind_all --port 8080

2024-09-28 16:32:34.618947: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-28 16:32:34.645833: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

TensorBoard 2.15.2 at http://default:8080/ (Press CTRL+C to quit)
^C
